<a href="https://colab.research.google.com/github/alexabruck/speculative-datasets/blob/master/Famous_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Famous trees

In [ ]:
!pip install SPARQLWrapper

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON

ENDPOINT_URL = "https://query.wikidata.org/sparql"

def get(query):
    user_agent = "Speculative Datasets (https://github.com/alexabruck/speculative-datasets)"
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(ENDPOINT_URL, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

query = """SELECT ?item ?itemLabel ?pic
WHERE 
{
  ?item wdt:P31 wd:Q10884. # Must be a tree
  ?item wdt:P18 ?image_. #has pic
  BIND(REPLACE(wikibase:decodeUri(STR(?image_)), "http://commons.wikimedia.org/wiki/Special:FilePath/", "") AS ?imageFileName_)
  BIND(REPLACE(?imageFileName_, " ", "_") AS ?imageFileNameSafe_)
  BIND(MD5(?imageFileNameSafe_) AS ?imageFileNameHash_)
  BIND(CONCAT("https://upload.wikimedia.org/wikipedia/commons/thumb/", SUBSTR(?imageFileNameHash_, 1 , 1 ), "/", SUBSTR(?imageFileNameHash_, 1 , 2 ), "/", ?imageFileNameSafe_, "/600px-", ?imageFileNameSafe_) AS ?pic)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # Helps get the label in your language, if not, then en language
}"""

response = get(query)
items = response["results"]["bindings"]
pics = [item['pic']['value'] for item in items]
print(len(pics))

553


## Download the pics

In [ ]:
!rm -rd pics

In [ ]:
!mkdir pics

for pic in pics:
  !wget $pic -P pics

In [ ]:
!zip -r results.zip pics
from google.colab import files
files.download("results.zip")